Pada notebook ini saya akan mendemonstrasikan proses **feature engineering** dalam rangka mengolah data mentah menjadi data yang siap digunakan untuk pemodelan machine learning. Data saya unduh dari database [PubChem](https://pubchem.ncbi.nlm.nih.gov/) berupa berkas **JSON** dengan rincian sebagai berikut:
- **En**: energi atomisasi molekul yang dihitung dengan metode medan gaya (*force-field*), nilai ini yang akan diprediksi oleh machine learning nantinya.
- **atoms**: berisi nama unsur serta posisinya pada molekul (dalam sistem koordinat kartesius).
- **Id**: nomor identitas molekul pada database PubChem.
- **shapeM**: bentuk multipole.

In [1]:
!wget --no-check-certificate \
    https://simpan.ugm.ac.id/s/VSejOaiJRfp30Mk/download \
    -O /tmp/predict-molecular-properties.zip

--2020-09-20 14:19:52--  https://simpan.ugm.ac.id/s/VSejOaiJRfp30Mk/download
Resolving simpan.ugm.ac.id (simpan.ugm.ac.id)... 175.111.88.197
Connecting to simpan.ugm.ac.id (simpan.ugm.ac.id)|175.111.88.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 95524990 (91M) [application/zip]
Saving to: ‘/tmp/predict-molecular-properties.zip’

/tmp/predict-molecu 100%[===================>]  91.10M  11.6MB/s    in 2m 4s   

2020-09-20 14:21:59 (749 KB/s) - ‘/tmp/predict-molecular-properties.zip’ saved [95524990/95524990]



In [2]:
import zipfile, os
import json

local_zip = '/tmp/predict-molecular-properties.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/predict-molecular-properties')
zip_ref.close()
base_dir = '/tmp/predict-molecular-properties/'
with open('/tmp/predict-molecular-properties/pubChem_p_00000001_00025000.json') as f:
    data = json.load(f)

[Rupp dkk (2012)](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.108.058301) mengusulkan definisi **Matriks Coulomb** yang hingga saat ini sangat populer digunakan sebagai fitur untuk mengembangkan machine learning untuk memprediksi energi atomisasi molekul. Secara matematis, Matriks Coulomb didefinisikan sebagai berikut:

$$
C_{IJ} = 
\begin{cases}
0.5 Z_I^{2.4} \text{ jika } I = J \\
\frac{Z_I Z_J}{|\textbf{R}_I - \textbf{R}_J|} \text{ jika } I \neq J
\end{cases} 
$$

dengan **Z** adalah muatan dan **R** adalah posisi atom dalam koordinat kartesian. Elemen dari Matriks Coulomb ini yang akan menjadi fitur (*predictor*) dalam model machine learning yang akan digunakan. Namun karena Matriks Coulomb tersebut simetris, maka hanya **elemen segitiga atas** yang akan diubah menjadi kolom-kolom pada **DataFrame**. Berikut adalah proses *feature engineering* yang saya lakukan untuk mengkonstruksi data mentah menjadi dataset yang siap digunakan (dalam bentuk berkas CSV) berdasarkan definisi Matriks Coulomb sebelumnya.

Mula-mula, saya mendefinisikan suatu **dictionary** yang menyimpan nilai dari lambang unsur, nomor atom, dan nomor massa dari masing-masing unsur yang merupakan unsur penyusun molekul yang akan dikonstruksi menjadi sebuah DataFrame. Proses ini juga sekaligus salah satu seleksi unsur penyusun molekul, dimana nantinya DataFrame yang terbentuk hanya berisi molekul-molekul dengan unsur penyusun yang disebutkan pada dictionary.

In [3]:
periodic_table = {'H':[1, 1.0079],
                  'C':[6, 12.0107],
                  'N':[7, 14.0067],
                  'O':[8, 15.9994],
                  'S':[16, 32.065],
                  'F':[9, 18.9984],
                  'Si':[14, 28.0855],
                  'P':[15, 30.9738],
                  'Cl':[17, 35.453],
                  'Br':[35, 79.904],
                  'I': [53, 126.9045]}

Saya membatasi hanya akan memilih molekul dengan jumlah atom paling banyak 50 (dapat diubah kemudian), artinya dimensi dari Matriks Coulomb adalah $50 \times 50$ sehingga dimensi fitur akan menjadi $\frac{50 \times 51}{2}$ (karena yang diambil hanya elemen segitiga atas). Untuk molekul dengan atom kurang dari 50, elemen Matriks Coulomb diisi dengan nilai 0 (*zero padding*). Berikut adalah *code*-nya.

In [4]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform

natMax = 50 # jumlah atom maksimum
nMolecules = len(data)

data_CM = np.zeros((nMolecules, natMax * (natMax + 1) // 2), dtype=float)
data_ids = np.zeros(nMolecules, dtype=int)
data_multipoles = np.zeros((nMolecules, 14), dtype=float)
data_mmff94 = np.zeros(nMolecules, dtype=float)

ind = 0
for molecule in data:
  natoms = len(molecule['atoms'])
  if natoms > natMax:
    continue
  data_mmff94[ind] = molecule['En']
  data_multipoles[ind, :] = molecule['shapeM']
  data_ids[ind] = molecule['id']

  full_CM = np.zeros((natMax, natMax))
  full_Z = np.zeros(natMax)

  pos = []
  Z = []
  for i, at in enumerate(molecule['atoms']):
    Z.append(periodic_table[at['type']][0])
    pos.append(at['xyz'])

  pos = np.array(pos, dtype=float)
  Z = np.array(Z, dtype=float)

  tiny = 1e-20

  dm = pdist(pos)

  coulomb_matrix = np.outer(Z, Z) / (squareform(dm) + tiny)
  full_CM[0:natoms, 0:natoms] = coulomb_matrix
  full_Z[0:natoms] = Z

  iu = np.triu_indices(natMax, k=1)
  coulomb_vector = full_CM[iu]

  shuffle = np.argsort(-coulomb_vector)
  coulomb_vector = coulomb_vector[shuffle]

  coulomb_matrix = squareform(coulomb_vector)
  assert np.trace(coulomb_matrix) == 0, 'Matriks Coulomb tidak sesuai'

  coulomb_matrix += 0.5 * np.power(full_Z, 2.4) * np.eye(natMax)
  iu = np.triu_indices(natMax)
  feature_vector = coulomb_matrix[iu]
  assert feature_vector.shape[0] == natMax * (natMax + 1) // 2, 'Dimensi fitur tidak sesuai'

  data_CM[ind] = feature_vector

  ind += 1

dat = np.column_stack((data_CM, data_multipoles))
df = pd.DataFrame(dat)

numfeats = np.shape(dat)[1]
cols = [x for x in range(1, numfeats + 1, 1)]
col_names = list(map(lambda x: 'f' + str(x), cols))
df.columns = col_names

df.insert(0, 'pubchem_id', data_ids)
df['En'] = data_mmff94

Berikut adalah hasil *feature engineering* yang telah dilakukan, telah terbentuk suatu DataFrame yang memuat informasi mengenai nomor identitas pada kolom **pubchem_id**, energi atomisasi molekul pada kolom **En**, serta elemen Matriks Coulomb pada kolom lainnya (kolom f1 dst).

In [5]:
df.head()

,pubchem_id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,...,f1251,f1252,f1253,f1254,f1255,f1256,f1257,f1258,f1259,f1260,f1261,f1262,f1263,f1264,f1265,f1266,f1267,f1268,f1269,f1270,f1271,f1272,f1273,f1274,f1275,f1276,f1277,f1278,f1279,f1280,f1281,f1282,f1283,f1284,f1285,f1286,f1287,f1288,f1289,En
0,1,73.516695,39.230740,38.058812,37.934254,35.169765,33.617995,28.032659,27.705090,27.600070,27.597789,27.596971,27.595640,24.020316,23.564754,23.533001,23.446742,22.167663,20.494977,20.395532,20.122970,20.057490,19.936149,19.832474,18.127966,17.634050,17.234416,17.209627,16.706301,16.681098,15.829653,15.348111,15.235092,14.486721,14.485699,14.485673,14.485580,14.485365,14.485352,14.331067,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,259.66,4.28,3.04,1.21,1.75,2.55,0.16,-3.13,-0.22,-2.18,-0.56,0.21,0.17,0.09,37.8010
1,2,73.516695,39.230539,39.213259,35.384864,35.171945,33.606458,28.271514,28.154189,27.704183,27.373063,27.372585,27.370423,24.021123,23.846171,23.593828,23.556082,22.936092,20.497956,20.119256,20.119146,19.936332,19.904420,19.855337,19.336183,18.951892,17.242046,17.057426,16.632489,15.921896,15.663035,15.630390,15.448626,15.234400,14.875074,14.623959,14.486121,14.485677,14.485434,14.484971,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,259.66,4.36,2.60,1.23,3.54,0.81,0.05,-0.40,-0.71,-2.51,-0.17,0.01,-0.33,-0.13,44.1107
2,3,73.516695,39.420140,35.571151,33.601772,33.564165,27.985222,26.861051,26.852468,24.963419,24.573904,23.933917,23.920948,23.245160,21.912838,20.864148,20.249226,20.212779,20.192761,19.787120,19.745225,18.126346,18.014741,17.232229,15.468289,15.168194,14.982951,14.884761,14.839441,14.419891,14.386142,14.379722,14.378173,14.312595,14.310079,14.068406,14.064434,14.017868,13.711428,13.575796,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,202.90,3.34,1.75,0.89,1.33,0.23,0.02,0.08,-0.70,-0.26,0.20,-0.29,-0.35,-0.34,19.4085
3,4,73.516695,33.629161,28.836604,23.630536,23.518647,20.151961,19.963944,16.945914,15.112884,14.125780,13.881916,8.227236,6.869758,6.869091,5.485307,5.475884,5.475851,5.473157,5.472439,5.472278,3.911906,3.341779,3.340884,3.071947,3.070200,3.038911,3.016245,3.001998,2.946448,2.942208,2.790443,2.776333,2.773886,2.770550,2.763733,2.762386,2.749930,2.539021,2.527455,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,92.05,1.98,1.20,0.63,0.77,0.25,0.00,-0.04,0.01,-0.48,-0.08,0.00,0.01,0.07,-0.1086
4,5,332.344906,79.959785,74.207114,74.205076,74.014645,38.987518,34.468622,33.988388,28.915828,27.885300,25.243490,25.233319,25.134095,24.334111,24.312040,24.289265,23.855993,23.839342,23.742536,22.864782,21.478403,20.078725,19.932399,19.927994,17.560199,17.360114,16.606425,16.158313,16.125264,13.965020,12.925481,12.919451,12.654280,12.563962,12.251966,11.679165,11.055120,11.015682,10.988146,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,175.32,6.18,1.12,0.84,2.42,0.15,0.00,0.07,-0.03,-0.25,0.00,-0.63,-0.27,0.01,-23.8799


Dapat juga dilihat jumlah molekul yang ada pada DataFrame dengan melihat jumlah baris pada DataFrame.

In [6]:
df.shape

(18205, 1291)

Selanjutnya DataFrame dapat disimpan pada berkas CSV untuk kemudian digunakan dalam proses modelling machine learning.

In [7]:
path = 'molecules_' + '1' + '_' + '25000' + '.csv'
df.to_csv(path)

Secara umum, ide utama pada penelitian ini adalah memanfaatkan definisi Matriks Coulomb untuk membangun fitur yang akan digunakan untuk membuat model machine learning dalam upaya memprediksi energi atomisasi molekul. Pada notebook berikutnya saya akan mendemonstrasikan proses pemodelan machine learning-nya.



---

Referensi

1. Himmetoglu B.: Tree based machine learning framework for predicting ground state energies of molecules, J. Chem. Phys 145, 134101 (2016) <br>
2. Halgren TA. Merck Molecular Force Field: I. Basis, Form, Scope, Parameterization and Performance of MMFF94. J. Comp. Chem. 1996;17:490-519. <br>
3. Halgren TA. Merck Molecular Force Field: VI. MMFF94s Option for Energy Minimization Studies. J. Comp. Chem. 1999;20:720-729. <br>
4. Kim, Sunghwan, Evan E Bolton, and Stephen H Bryant. “PubChem3D: Shape Compatibility Filtering Using Molecular Shape Quadrupoles.” J. Cheminf. 3 (2011): 25. <br>
5. Matthias Rupp, Alexandre Tkatchenko, Klaus-Robert Müller, O. Anatole von Lilienfeld. Fast and Accurate Modeling of Molecular Atomization Energies with Machine Learning, Phys. Rev. Lett. 108, 058301 (2012)